In [1]:
%load_ext tuna

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import numpy as np
import dill
import os
import seaborn as sns
import torch
import pandas as pd

from collections import namedtuple
from hmmlearn.hmm import MultinomialHMM
from scipy.special import logsumexp
from scipy.stats import multivariate_normal
from spn.algorithms.Inference import log_likelihood

from engine_dataset import generate_engine_dataset, generate_engine_datasets

from oSLRAU_run import get_data
from spn.structure.leaves.parametric.Parametric import Gaussian, Bernoulli, Categorical,  In_Latent, Multivariate_Gaussian
from spn.structure.Base import Context
from spn.algorithms.LearningWrappers import learn_parametric
from spn.io.Graphics import plot_spn
import numpy as np
from spn.algorithms.Inference import log_likelihood
from sklearn.model_selection import train_test_split
from spn.algorithms.oSLRAU import oSLRAU, oSLRAUParams
from spn.algorithms.RSPN import RSPN
from spn.algorithms.TransformStructure import Prune ,Prune_oSLRAU

from hmmlearn import hmm
from spn.algorithms.Inference import log_likelihood



from experiment_utils.datasets.vasopressors_dataset import load_vasopressors_dataset

In [164]:
SEQ_LENGTH = 6
NUM_STATES = 20
NUM_SEEDS = 5
SEEDS = range(42, 42 + NUM_SEEDS)

train_vaso_datasets, test_vaso_datasets = load_vasopressors_dataset(
    seq_length=SEQ_LENGTH,
)
num_inputs = (
    int(
        np.max(train_vaso_datasets[0].inputs_raw)
    )
    + 1
)
num_observations = (
    int(
        np.max(train_vaso_datasets[0].observations_raw_discrete)
    )
    + 1
)

  0%|          | 0/6052 [00:00<?, ?it/s]

Loaded 5990 data points
Partitioning into folds...
Done!


In [173]:
# unique values in the observations_raw_discrete
train_ds_0 = train_vaso_datasets[0]

n=0
for row in train_ds_0.inputs_raw:
    if 1 in row:
        print(row)
        n+=1

print(n)




# for vaso in train_vaso_datasets:
#     print(np.unique(vaso.observations_raw_discrete))


[0 0 0 0 1 1]
[0 0 0 0 1 1]
[0 1 1 1 1 1]
[0 0 0 0 0 1]
[0 0 0 1 1 1]
[0 0 1 1 1 1]
[0 0 0 0 1 1]
[0 0 0 0 1 1]
[0 1 1 1 1 1]
[0 0 1 1 1 1]
[0 0 0 0 0 1]
[0 0 1 1 1 1]
[0 0 0 1 1 1]
[0 0 0 1 1 0]
[0 0 0 0 0 1]
[0 1 1 1 1 1]
[0 0 0 0 0 1]
[0 1 0 1 1 1]
[0 0 0 0 0 1]
[0 0 1 1 1 1]
[0 1 1 1 1 1]
[0 0 0 0 0 1]
[0 1 1 1 1 1]
[0 0 1 1 1 1]
[0 0 0 0 0 1]
[0 0 1 0 0 0]
[0 1 1 1 1 1]
[0 0 0 0 1 0]
[0 0 0 0 1 1]
[0 1 1 1 1 1]
[0 0 0 1 1 1]
[0 0 0 1 1 1]
[0 0 1 0 0 0]
[0 0 0 1 1 1]
[0 0 0 1 1 1]
[0 0 0 0 1 0]
[0 0 0 1 1 1]
[0 0 0 0 1 1]
[0 0 0 1 1 1]
[0 0 0 0 0 1]
[0 1 1 1 1 1]
[0 0 0 0 1 1]
[0 0 1 1 1 1]
[0 0 0 0 0 1]
[0 1 0 0 0 0]
[0 1 1 1 1 1]
[0 0 1 1 1 1]
[0 0 0 0 0 1]
[0 0 0 0 1 1]
[0 0 0 0 1 1]
[0 0 0 0 0 1]
[0 0 0 1 1 1]
[0 0 0 1 1 1]
[0 0 1 1 1 1]
[0 0 0 0 1 1]
[0 1 1 1 1 1]
[0 0 0 0 1 1]
[0 0 0 0 1 1]
[0 0 1 1 1 1]
[0 0 1 1 1 1]
[0 1 1 1 1 1]
[0 1 1 1 1 1]
[0 1 1 1 1 1]
[0 0 1 1 1 1]
[0 0 0 1 1 1]
[0 0 1 1 1 1]
[0 0 0 0 0 1]
[0 0 1 1 1 1]
[0 0 0 0 0 1]
[0 0 0 0 0 1]
[0 1 1 0 0 0]
[0 0 0

In [162]:
train_vaso_0 = train_vaso_datasets[0]

for i, s in enumerate(SEEDS):
    print(f"_______{s}______\n")
    ds = train_vaso_datasets[i]
    
    print(np.unique(ds.inputs_raw))

_______42______

[0 1]
_______43______

[0 1]
_______44______

[0 1]
_______45______

[0 1]
_______46______

[0 1]


In [5]:
print(train_vaso_0.observations_raw_discrete.shape)
print(train_vaso_0.inputs_raw.shape)

(4792, 6)
(4792, 6)


In [188]:
def train_rspn_vaso(train_data, test_data, epoch, rspn=None, os_args=None, do_updates=True):

    if len(train_data.shape) == 3: train_data = np.squeeze(train_data)
    if len(test_data.shape) == 3: test_data = np.squeeze(test_data)

    if rspn is None:
    
        rspn = RSPN(num_variables=os_args["num_variables"], num_latent_variables=os_args["num_latent_variables"], num_latent_values=os_args["num_latent_values"])
        first_mini_batch = train_data[0:os_args["mini_batch_size"]]
        n = first_mini_batch.shape[1]

        print(f"n: {n}") # = num_time_steps_mix * n_dim
        
        context = [Categorical, Bernoulli]*(int(n/2)) # Bernoulli Categorical
        ds_context = Context(parametric_types=context).add_domains(first_mini_batch[:, :os_args["num_variables"]])
        spn, initial_template_spn, top_spn = rspn.build_initial_template(first_mini_batch, ds_context, os_args["len_sequence_varies"])

    
    no_of_minibatches = int(train_data.shape[0] / os_args["mini_batch_size"])

    print(f"no of minibatches: {no_of_minibatches}")

    
    for i in range(1, no_of_minibatches):
        mini_batch = train_data[i * os_args["mini_batch_size"]: (i+1) * os_args["mini_batch_size"]]
    
        update_template = False
        
        if i==10 and epoch == 0 and do_updates: #and epoch == 0:
            update_template = True

        
    
        template_spn = rspn.learn_rspn(mini_batch, update_template, os_args["os_params"], os_args["unroll"],
                                                os_args["full_update"], os_args["update_leaves"],
                                                os_args["len_sequence_varies"])
           
            
    test_ll = np.sum(rspn.log_likelihood(test_data, os_args["unroll"], os_args["len_sequence_varies"]))


    return test_ll, rspn, template_spn

In [189]:
def final_ll_vaso(data, num_epochs=1, do_plot_spn=True, os_args=None, split_data=False,rspn=None, do_updates=True):
    ll = -np.inf
    if split_data:
        train_data, val_data= train_test_split(data, test_size=0.1, random_state=42)
    
    else:
        train_data, val_data = data, data
    for epoch in range(num_epochs):
        print(train_data.shape)
        
        
        epoch_ll, rspn, template_spn = train_rspn_vaso(train_data, val_data, epoch, rspn, os_args, do_updates)        
        print(f"epoch_ll: {epoch_ll}")
        
        ll = epoch_ll
        del template_spn

    print(f" \n\n\n final ll: {ll}\n\n\n")

    if do_plot_spn:
        plot_spn(template_spn, 'rspn_final_template.pdf')
    
    return rspn

In [190]:
def process_ds(ds):
    raw_inputs = ds.inputs_raw
    obs_raw_cont = ds.observations_raw_discrete


    comb_ds = []
    ones_ds = []
    for i in range(raw_inputs.shape[0]):
        comb_ds_i = []
        for j in range(raw_inputs.shape[1]):
             comb_ds_j = [obs_raw_cont[i, j], raw_inputs[i,j]]
             comb_ds_i.append(comb_ds_j)


        comb_ds.append(comb_ds_i)

    comb_ds = np.array(comb_ds)
    comb_ds = comb_ds.reshape(raw_inputs.shape[0], -1)

    for row in comb_ds:
        if 1 in row:
            ones_ds.append(row)
    
    ones_ds = np.array(ones_ds)
    return comb_ds, ones_ds

In [191]:
n_states = 2
n_dim = 2

num_time_steps = 6

oSLRAU_params = oSLRAUParams(mergebatch_threshold=128, corrthresh=0.1, mvmaxscope=1, equalweight=True, currVals=True)

os_args = {
    "num_variables": num_time_steps * n_dim,
    "num_latent_variables" : 2,
    "num_latent_values" : 2,
    "unroll" : "backward",
    "full_update" : False,
    "update_leaves" : True,
    "len_sequence_varies" : False,
    "mini_batch_size" : 5,
    "update_after_no_min_batches" : 10,
    "do_updates" : False,
    "os_params": oSLRAU_params
    }

In [192]:
log_likelihoods = []
rspn_vaso_models = []

In [196]:
base_seed = SEEDS[0]
seed_dict = {SEEDS[i]: i for i in range(len(SEEDS))}
seed_ls = SEEDS # [SEEDS[1]]

for seed in seed_ls:
    i = seed_dict[seed]
    print(f"——————————[ Running experiment for seed {seed} ]——————————")
    train_ds, train_all_ones = process_ds(train_vaso_datasets[i])
    test_ds, _ = process_ds(test_vaso_datasets[i])
    
    rspn_vaso = final_ll_vaso(train_ds, num_epochs=3, do_plot_spn=False, os_args=os_args, rspn=None)
    rspn_vaso = final_ll_vaso(train_all_ones, num_epochs=10, do_plot_spn=False, os_args=os_args, rspn=rspn_vaso, do_updates=False)
    rspn_test_ll_mimic = np.sum(rspn_vaso.log_likelihood(test_ds, os_args["unroll"], os_args["len_sequence_varies"]))
    print(f"seed {seed}: log likelihood: {rspn_test_ll_mimic}")
    log_likelihoods.append(rspn_test_ll_mimic)
    rspn_vaso_models.append(rspn_vaso)

——————————[ Running experiment for seed 42 ]——————————
(4792, 12)
n: 12
==>> data.shape: (5, 12)
len(self.meta_types): 12
self.meta_types.values(): dict_values([<MetaType.DISCRETE: 3>, <MetaType.BINARY: 2>, <MetaType.DISCRETE: 3>, <MetaType.BINARY: 2>, <MetaType.DISCRETE: 3>, <MetaType.BINARY: 2>, <MetaType.DISCRETE: 3>, <MetaType.BINARY: 2>, <MetaType.DISCRETE: 3>, <MetaType.BINARY: 2>, <MetaType.DISCRETE: 3>, <MetaType.BINARY: 2>])
len(self.meta_types.values()): 12
data[:, col]: [4 0 4 0 6]
==>> domain_values: [0, 6]
feature_meta_type: MetaType.DISCRETE
len(self.meta_types.values()): 12
data[:, col]: [0 0 0 0 0]
==>> domain_values: [0, 0]
feature_meta_type: MetaType.BINARY
==>> domain_values: [0, 0]
len(self.meta_types.values()): 12
data[:, col]: [4 0 4 0 6]
==>> domain_values: [0, 6]
feature_meta_type: MetaType.DISCRETE
len(self.meta_types.values()): 12
data[:, col]: [0 0 0 0 0]
==>> domain_values: [0, 0]
feature_meta_type: MetaType.BINARY
==>> domain_values: [0, 0]
len(self.meta_ty

In [47]:
# print(f"Mean: {np.array(log_likelihoods).mean()}")
# print(f"STD: {np.array(log_likelihoods).std()}")

Mean: -14290.806510859877
STD: 1144.699024595064


In [197]:
print(f"Mean: {np.array(log_likelihoods).mean()}")
print(f"STD: {np.array(log_likelihoods).std()}")

Mean: -18062.228323318934
STD: 1488.367852715393


In [202]:
import pickle

test_ds_list = []
for ds in test_vaso_datasets:
    curr_ds, _ = process_ds(ds)
    test_ds_list.append(curr_ds)


with open('all_vaso_vars.pickle', 'wb') as file:
     pickle.dump(rspn_vaso_models[-5:], file)  # Store variable 'a' rspn_mimic_models
     pickle.dump(test_ds_list, file) 

In [195]:
# import pickle
# with open('all_vaso_vars.pickle', 'rb') as file:  
#     rspn_vaso_models = pickle.load(file)  
#     test_ds_list_old = pickle.load(file)

# NUM_SEEDS = 5
# SEEDS = range(42, 42 + NUM_SEEDS)

# len(rspn_vaso_models), len(test_ds_list_old)

(5, 5)

In [170]:
for ds in train_vaso_datasets:
    print(np.unique(ds.inputs_raw, return_counts=True))

(array([0, 1]), array([27221,  1531]))
(array([0, 1]), array([27269,  1483]))
(array([0, 1]), array([27312,  1440]))
(array([0, 1]), array([27321,  1431]))
(array([0, 1]), array([27805,   947]))
